In [1]:
import sys
sys.path.append("/p300s/wangmx_group/xutingfeng/SIS/")


In [2]:

from sis.dataset import SIS_MSADataset
import torch 
# import transformers 


# Step1 dataset load 
# device = try_gpu()
device = "cuda:1"

sisdataset = SIS_MSADataset(root_dir = "/p300s/wangmx_group/xutingfeng/SIS/data/msa_total_data.csv", device = device)
# sisdataset = SIS_MSADataset(root_dir = "/p300s/wangmx_group/xutingfeng/SIS/data/msa_total_data.csv", device = device)


aa_vocab = sisdataset.aa_vocab
sis_datasetDict = sisdataset.dataset_dict

train_dataloader = torch.utils.data.DataLoader(sis_datasetDict["train"], batch_size=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(sis_datasetDict["test"], batch_size=4, shuffle=True)




Using custom data configuration default-25cf8ae19bd6daf0
Reusing dataset csv (/home/xutingfeng/.cache/huggingface/datasets/csv/default-25cf8ae19bd6daf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-25cf8ae19bd6daf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-2ff8430349cd8a96.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-25cf8ae19bd6daf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-37eeedbc7dd983ee.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-25cf8ae19bd6daf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-fb14288b7f2c917c.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-25cf8ae19bd6daf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-281efcb2da39b9a3.arrow
Loading cached split indices for dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-25cf8ae19bd6daf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-389e93295146aaa7.arrow and /home/xutingfeng/.cache/huggingface/datasets/csv/default-25cf8ae19bd6daf0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-38b56db73156af48.arrow


In [3]:
import importlib 

import sis.model.sismodel as sismodel 

In [15]:
importlib.reload(sismodel)


<module 'sis.model.sismodel' from '/p300s/wangmx_group/xutingfeng/SIS/sis/model/sismodel.py'>

In [16]:
from sis.train import train
from torch import nn 

from torch.utils.tensorboard import SummaryWriter
import os.path as osp
import os 

from sis.dataset.EmbeddingData.parse import load_expasy_embedding_dict, load_meiler_embedding_dict
from sis.model.embedding import EmbeddingLayer, OnehotLayer

from sis.model.sismodel import Regression

from sis.utils import modelParametersNum, try_gpu
# Step2 build model
length = sisdataset.SLF_max_length + sisdataset.SRnase_max_length

embedding_layer = EmbeddingLayer(aa_embedding_dict=load_meiler_embedding_dict(), aa_vocab=aa_vocab)  # property embedding 
# embedding_layer = OnehotLayer(len(aa_vocab))  # onehot

model = Regression(EmbeddingLayer = embedding_layer) 

# Step3定义 optim and loss
optim = torch.optim.NAdam(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()
print(modelParametersNum(model))
## log path and save path
model_name = model.__class__.__name__
log_path = f"/p300s/wangmx_group/xutingfeng/SIS/output/logs/{model_name}"

summaryWriter = SummaryWriter(log_path)  # 使用tensorboard记录loss

save_path = f"/p300s/wangmx_group/xutingfeng/SIS/output/model"
# Step4 train
train(net = model, train_dataIter= train_dataloader, test_dataIter= test_dataloader,loss=loss, optim=optim, summary_writer = summaryWriter, epochs=100, device=device,
 savemodel=False, 
#  save_path = save_path
 )



模型总参数个数：169
169
epoch 10 : train: mean loss/per 1.848 acc is 0.667 and time is 497.4220289568612review/per sec
test: loss 0.924 and acc is 0.800
epoch 20 : train: mean loss/per 1.848 acc is 0.667 and time is 528.0946825030428review/per sec
test: loss 0.924 and acc is 0.800
epoch 30 : train: mean loss/per 1.848 acc is 0.667 and time is 475.3560378534595review/per sec
test: loss 0.924 and acc is 0.800
epoch 40 : train: mean loss/per 1.848 acc is 0.667 and time is 529.1922153569013review/per sec
test: loss 0.693 and acc is 0.800
epoch 50 : train: mean loss/per 1.848 acc is 0.667 and time is 532.1220186072737review/per sec
test: loss 0.924 and acc is 0.800
epoch 60 : train: mean loss/per 1.848 acc is 0.667 and time is 495.0313060412234review/per sec
test: loss 0.693 and acc is 0.800
epoch 70 : train: mean loss/per 1.848 acc is 0.667 and time is 527.1452001996935review/per sec
test: loss 0.924 and acc is 0.800
epoch 80 : train: mean loss/per 1.848 acc is 0.667 and time is 498.54210840847486

In [17]:
for i in train_dataloader:
    o = model(i, return_scores=True)
    print(o)
    print(model.scores)
    print(i["SLF_Seq_mask"].shape)
    break 

tensor([[0.4996],
        [0.4996],
        [0.4996],
        [0.4996]], device='cuda:1', grad_fn=<SigmoidBackward0>)
tensor([[1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09],
        [1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09],
        [1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09],
        [1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09, 1.0000e-09,
         1.0000e-09, 1.0000e-09]], device='cuda:1', grad_fn=<CatBackward0>)
torch.Size([4, 2161])


In [ ]:
from itertools import chain
import pandas as pd 
from sis.train import test, predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

iter_dataloader = train_dataloader

o = predict(iter_dataloader, model).cpu()
label = torch.concat([i["label"] for i in iter_dataloader]).cpu()

def flatten_list(x:list):
    return list(chain.from_iterable(x))
names = flatten_list([zip(i["SLF"], i["SRnase"]) for i in iter_dataloader])

pred = (o>0.5).int().flatten()
# pred = o.flatten()

pd.DataFrame({"names":names, "pred":pred, "label":label})

In [5]:
x = torch.normal(0,1, (2, 4, 3))
x

tensor([[[ 0.3495,  0.1740, -1.9586],
         [-0.3947, -0.0043,  0.2500],
         [-0.2303,  1.7879,  1.0129],
         [-0.4710, -0.6035, -1.2733]],

        [[ 0.5422,  0.1914, -1.4903],
         [-0.8044,  0.1609,  1.0191],
         [ 0.4501, -1.0653,  1.7319],
         [ 1.9115,  0.3837,  2.0211]]])

In [7]:
select = torch.tensor([[True, False, False, False], [False, True, True, False]])
select


tensor([[ True, False, False, False],
        [False,  True,  True, False]])

In [9]:
def torch_mean(x, select):
    # True 会被考虑，False会被丢掉
    # select 和x维度应该数量一直，保证可以广播机制
    # x = (batch, N1, N2..) select = (batch, N1, ...) bool matrix

    select = select.unsqueeze(-1)
    select = ~select  
    x = x.masked_fill(select,1e-9).sum(dim=1)

    count = select.int().sum(1)

    return x/count 

    

In [10]:
o = torch_mean(x, select)
o

tensor([[ 0.1165,  0.0580, -0.6529],
        [-0.1771, -0.4522,  1.3755]])

In [112]:
oo.requires_grad

False

In [81]:
select_2d.shape

torch.Size([2, 4])

TypeError: bad operand type for unary ~: 'list'

In [90]:
f = ~select_2d

In [91]:
f

tensor([[False,  True,  True,  True],
        [ True, False, False,  True]])